In [11]:
import pandas as pd
import numpy as np
import pickle

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm, grid_search
import time
import pickle


In [ ]:
ubdc = pd.read_csv('data', encoding='latin1', nrows=20)
print(list(ubdc))
print(ubdc.dtypes)
'''ubdc.to_csv('data')'''

In [ ]:
wf= pd.read_csv('data', encoding='latin1', usecols=['udprn', 'id'])
print(list(wf))

In [ ]:
print(wf.id.head())
print(len(wf))
print(ubdc.listing_id.head())

In [27]:
uniq__value_list = wf.id.unique().tolist()

In [ ]:
chunksize = 10**5
chunks = 0
start = time.time() 
start2 = time.time() 
yes = 0
no = 0
UBDC = "data"
cols = ['cols']
# Ensures that only the header for the first chunk is kept (as all the files will be appended)
writeHeader = True
# START: For each chunk in zoopla data:        
for chunk in pd.read_csv(UBDC, encoding='latin1',chunksize=chunksize, usecols=cols):
    #increment count
    chunks +=1
    
    start_chunk = time.time()
    
    df_yes = chunk[chunk['listing_id'].isin(uniq__value_list)]
    yes+= len(df_yes)
    df_no = chunk[~chunk['listing_id'].isin(uniq__value_list)]
    no+= len(df_no)
    
    # Print info after each chunk to keep up to date with progress
    end = time.time() - start_chunk
    interim = (time.time() - start2)//60
    print("\n ****** \n This iteration :\t {} \n Time taken (Seconds):   \t {} \n Chunk number (out of 228): \t {}\n Interim time (Minutes): \t {} \n Total df_yes: \t {} \n Total df_no: \t {} \n   ****** \n" .format(' ',end, chunks, interim, yes, no))
    
    # 3. Write to csv and tell whether or not to include the header
    if writeHeader is True:    
        df_no.to_csv('filepath', mode='a', header=True, index=False)
        df_yes.to_csv('filepath', mode='a', header=True, index=False)
        writeHeader = False
    else:
        df_no.to_csv('filepath', mode='a', header=False, index=False)
        df_yes.to_csv('filepath', mode='a', header=True, index=False) 

print("\n ****** \n Chunks Processed: \t {}\n Time elapsed (Minutes):  \t{}\n ****** \n".format(chunks, (time.time() - start)//60))

In [51]:
'''with open('wf_label.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    preds = pickle.load(f)
'''

with open('clf.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    clf = pickle.load(f)
with open('trigram_LR.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    trigram_LR = pickle.load(f)


'''    
with open('train_X.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    train_X = pickle.load(f)
with open('train_y.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    train_y = pickle.load(f)
'''

with open('X.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X_train = pickle.load(f)
with open('y.pickle', 'rb') as f:
    # The protocol version used isdetected automatically, so we do not
    # have to specify it.
    y_train = pickle.load(f)

### Define functions to get text in right format

In [52]:
def removeStopWords(input):
    exclude = set(string.punctuation)
    output = ' '.join([word for word in input.split() if word not in stopwords.words("english")])
    output = ''.join(ch for ch in output if ch not in exclude)
        
    return pd.Series(dict(output=output))

def remove_non_ascii (text):
    return ''.join(i for i in text if ord(i)<128)

# Stem and lemmatize
porter_stemmer = PorterStemmer()
wnl = WordNetLemmatizer()

### String cleaning

In [ ]:
chunksize = 10**5
chunks = 0
start = time.time() 
start2 = time.time() 
ubdc = 'data'
cols = ['cols']
# Ensures that only the header for the first chunk is kept (as all the files will be appended)
writeHeader = True
# START: For each chunk in zoopla data:        
for chunk in pd.read_csv(ubdc, chunksize=chunksize, usecols=cols):
    #increment count
    chunks +=1
    
    start_chunk = time.time()
    
    # 1.  SOME CLEANING OF DESCRIPTION    
    chunk['description'] = chunk['description'].astype(str)
    chunk["description"] = chunk["description"].str.lower()   
    ##  Shouldn't really use regex to parse html, should use  parser, but this was done for quickness
    chunk['description'] = chunk['description'].str.replace('\n', ' ') # this one works
    chunk['description'] = chunk['description'].str.replace('&amp;pound;', '£')
    chunk['description'] = chunk['description'].str.replace('&amp;', '&')
    chunk['description'] = chunk['description'].str.replace('&amp;amp;', '&')
    chunk['description'] = chunk['description'].str.replace('&amp;quot;', '"') 
    chunk['description'] = chunk['description'].str.replace('&amp;nbsp;', ' ')
    chunk['description'] = chunk['description'].str.replace('&amp;#39;', "'")
    chunk['description'] = chunk['description'].str.replace('&#39;', "'")
    chunk['description'] = chunk['description'].str.replace('&#x2019;', "'")
    chunk['description'] = chunk['description'].str.replace('&nbsp;', "")
    chunk['description'] = chunk['description'].str.replace('&#231;', "c")
    chunk['description'] = chunk['description'].str.replace('&#x2018', "'")
    chunk['description'] = chunk['description'].str.replace('&#8217;', "'")
    chunk['description'] = chunk['description'].str.replace('&quot;', '"')
    chunk['description'] = chunk['description'].str.replace('&#x2013;', '') # meant to be a dash
    chunk['description'] = chunk['description'].str.replace('.&#13;&#10;', " ")
    # Apply the function to remove special characters
    chunk['description'] = chunk['description'].apply(lambda x : remove_non_ascii(x))
    
    print('Next step')
    # 2. Start removing stop words, lematizing etc
    # 2a. Stop words
    print('stopwords')
    chunk['ml_text'] = chunk['description'].apply(lambda x: removeStopWords(x))
    # 2b. Lematizing etc
    print('lemma')
    chunk['token_text'] = chunk['ml_text'].apply(lambda x : x.split(" "))
    chunk['stem_text'] = chunk['token_text'].apply(lambda x : [porter_stemmer.stem(y) for y in x])
    chunk['stem_text'] = chunk['stem_text'].apply(lambda x : ' '.join(x))
    # Drop columns no longer needed
    chunk.drop(['description', 'ml_text', 'token_text'], axis=1, inplace=True) 
    
    
    # Print info after each chunk to keep up to date with progress
    end = time.time() - start_chunk
    interim = (time.time() - start2)//60
    print("\n ****** \n This iteration :\t {} \n Time taken (Seconds):   \t {} \n Chunk number (out of 228): \t {}\n Interim time (Minutes): \t {} \n  ****** \n" .format(' ',end, chunks, interim))
    
    # 3. Write to csv and tell whether or not to include the header
    if writeHeader is True:    
        chunk.to_csv('filepath', mode='a', header=True, index=False)
        writeHeader = False
    else:
        chunk.to_csv('filepath', mode='a', header=False, index=False)

print("\n ****** \n Chunks Processed: \t {}\n Time elapsed (Minutes):  \t{}\n ****** \n".format(chunks, (time.time() - start)//60))


### Perform machine learning

In [ ]:
chunksize = 10**5
chunks = 0
start = time.time() 
start2 = time.time() 
ubdc = 'data'
cols = ['cols']
# Ensures that only the header for the first chunk is kept (as all the files will be appended)
writeHeader = True
# START: For each chunk in zoopla data:        
for chunk in pd.read_csv(ubdc, chunksize=chunksize, usecols=cols):
    #increment count
    chunks +=1
    
    start_chunk = time.time()
    
    X=chunk['stem_text'].values.astype('U')
    trigram_LR.fit(X_train, y_train)
    predictions= trigram_LR.predict(X)
    y_pred_prob = trigram_LR.predict_proba(X)[:,1]
    X_LR_output = pd.DataFrame({'X' :, 'predictions':predictions, 'pred_prob' :y_pred_prob, 'unique_id' :chunk['unique_id'],'lat':chunk['LATITUDE'], 'long':chunk['LONGITUDE']})
    #LR_output = pd.DataFrame({'X':X, 'predicted':y_pred_prob, 'listing_id':df_A['listing_id'], 'lat':df_A['lat'], 'long':df_A['long'] })
    sout = X_LR_output.sort_values(['X', 'predictions'], ascending=[False, False])
    
    
    
    # Print info after each chunk to keep up to date with progress
    end = time.time() - start_chunk
    interim = (time.time() - start2)//60
    print("\n ****** \n This iteration :\t {} \n Time taken (Seconds):   \t {} \n Chunk number (out of 228): \t {}\n Interim time (Minutes): \t {} \n  ****** \n" .format(' ',end, chunks, interim))
    
    # 3. Write to csv and tell whether or not to include the header
    if writeHeader is True:    
        sout.to_csv('filepath', mode='a', header=True, index=False)
        writeHeader = False
    else:
        sout.to_csv('filepath', mode='a', header=False, index=False)

print("\n ****** \n Chunks Processed: \t {}\n Time elapsed (Minutes):  \t{}\n ****** \n".format(chunks, (time.time() - start)//60))